In [1]:
#!python3 -m pip install owlready2

In [4]:
import owlready2
from owlready2 import get_ontology
from owlready2.namespace import SOME, ONLY
import csv

In [6]:
foodon = get_ontology("../../../capstone/ontologies/foodon_filtered.owl").load()

In [7]:
counter = 0
for cls in foodon.classes():
    counter += 1
    print(cls)
    if counter > 10:
        break

obo.FOODON_00002403
obo.FOODON_00001002
Thesaurus.C103180
obo.OBI_0000658
obo.AGRO_00000025
obo.AGRO_00002071
obo.AGRO_00000322
obo.AGRO_00000360
obo.ENVO_01000408
obo.AGRO_00000363
obo.OBI_0000011


In [8]:
foodon.get_namespace(foodon.base_iri)

get_ontology("http://purl.obolibrary.org/obo/foodon.owl/")

The function below can be cleaned up (repeats itself) and doesn't handle all cases.

In [9]:

def create_nodes_relations(
    ontology,
    foodon_nodes_file,
    and_nodes_file,
    blank_nodes_file,
    ff_relations_file,
    fa_relations_file,
    ba_relations_file,
    bf_relations_file,
):
    #relationship and node headers
    # Three types of node files.
    foodon_nodes_header = ["foodonId:ID(FoodOn)", "foodon_label", ":LABEL"]
    foodon_nodes = {}

    and_nodes_header = ["andId:ID(And)", "type", ":LABEL"]
    and_nodes = {}
    and_counter = 0

    blank_nodes_header = ["blankId:ID(Blank)", "type", ":LABEL"]
    blank_nodes = {}
    blank_counter = 0

    # Many relations files:
    # FoodOn node to FoodOn node relations
    ff_relations_header = [
        ":START_ID(FoodOn)",
        ":END_ID(FoodOn)",
        ":TYPE",
        "label:IGNORE",
        "restriction_type:IGNORE",
    ]
    ff_relations = []

    # FoodOn node to And node relations
    fa_relations_header = [
        ":START_ID(FoodOn)",
        ":END_ID(And)",
        ":TYPE",
        "label:IGNORE",
        "restriction_type:IGNORE",
    ]
    fa_relations = []

    # Blank node to And node relations
    ba_relations_header = [
        ":START_ID(Blank)",
        ":END_ID(And)",
        ":TYPE",
        "label:IGNORE",
        "restriction_type:IGNORE",
    ]
    ba_relations = []

    # Blank node to FoodOn node relations (can have restrictions)
    bf_relations_header = [
        ":START_ID(Blank)",
        ":END_ID(FoodOn)",
        ":TYPE",
        "label",
        "restriction_type",
    ]
    bf_relations = []

    # Loop through all classes.
    for cls in ontology.classes():

        # Handle this class (as long as it has a label, add it to the list of foodon_nodes)
        # list(set(cls.label)) is used since sometimes labels are repeated.
        try:
            if len(list(set(cls.label))) != 0:
                if not str(cls) in foodon_nodes:
                    node = [str(cls), list(set(cls.label), "FoodOn")]
                    foodon_nodes[str(cls)] = node
        except:
            pass

        # Check for is_a relations.
        for c in set(cls.is_a):

            # If the is_a relation points to just another class, treat it as above:
            # (as long as it has a label, add it ot the list of foodon_nodes).
            # Also add the "is_a" relation to the relations.
            if isinstance(c, owlready2.entity.ThingClass):
                try:
                    if len(list(set(c.label))) != 0:
                        if not str(c) in foodon_nodes:
                            node = [str(c), list(set(c.label))]
                            foodon_nodes[str(c)] = node
                        if (str(cls) in foodon_nodes) and (str(c) in foodon_nodes):
                            ff_relations.append([str(cls), str(c), "is_a", "", ""])
                except:
                    pass

            # Handle the case where the is_a relation points to an "And" object:
            # Add to the and_nodes and relations.
            elif isinstance(c, owlready2.class_construct.And):
                and_counter += 1
                and_nodes[f"AND_{and_counter}"] = [f"AND_{and_counter}", "", "and"]
                if str(cls) in foodon_nodes:
                    fa_relations.append(
                        [str(cls), f"AND_{and_counter}", "is_a", "", ""]
                    )

                # For each of the subclasses in the "And" class, add a BLANK_x node,
                # and record the relation.
                # We also add the foodon object within the restriction to the list of foodon_nodes
                # and the "member_of" relations.
                for sub_c in c.Classes:
                    blank_counter += 1
                    blank_nodes[f"BLANK_{blank_counter}"] = [
                        f"BLANK_{blank_counter}",
                        "",
                        "blank",
                    ]
                    ba_relations.append(
                        [
                            f"BLANK_{blank_counter}",
                            f"AND_{and_counter}",
                            "member_of",
                            "",
                            "",
                        ]
                    )

                    # It looks like there can be more than one "layer" of "And" relations:
                    #
                    #                    foodon_node
                    #                       |
                    #                    AND_1
                    #                   /     \
                    #              AND_2     BLANK_1
                    #            /     \          |
                    #        BLANK_2   BLANK_3  foodon_node
                    #         |           |
                    #     foodon_node  foodon_node

                    # Ignoring this second (or deeper) layer for now.
                    if isinstance(sub_c, owlready2.class_construct.And):
                        continue

                    # Ignoring Or relations.
                    if isinstance(sub_c, owlready2.class_construct.Or):
                        continue

                    # Ignoring Not relations.
                    if isinstance(sub_c, owlready2.class_construct.Not):
                        continue

                    # Checking to make sure the restriction type is "SOME"
                    if sub_c.type == SOME:
                        restriction_type = "some"
                    elif sub_c.type == ONLY:
                        restriction_type = "only"

                    # Ignoring (seemingly) invalid restriction types.
                    elif isinstance(sub_c.type, list):
                        continue

                    if str(sub_c.value) in foodon_nodes:
                        bf_relations.append(
                            [
                                f"BLANK_{blank_counter}",
                                str(sub_c.value),
                                str(sub_c.property),
                                list(set(sub_c.property.label)),
                                restriction_type,
                            ]
                        )
                    if str(sub_c.property) in foodon_nodes:
                        node = [
                            str(sub_c.property),
                            list(set(sub_c.property.label)),
                            "FoodOn",
                        ]
                        foodon_nodes[str(sub_c.property)] = node

        # Check for equivalent_to relations.
        for c in set(cls.equivalent_to):

            # If the is_a relation points to just another class, treat it as above:
            # (as long as it has a label, add it ot the list of foodon_nodes).
            # Also add the "equivalent_to" relation to the relations.
            if isinstance(c, owlready2.entity.ThingClass):
                try:
                    if len(list(set(c.label))) != 0:
                        if not str(c) in foodon_nodes:
                            node = [str(c), list(set(c.label))]
                            foodon_nodes[str(c)] = node
                        if (str(cls) in foodon_nodes) and (str(c) in foodon_nodes):
                            ff_relations.append(
                                [str(cls), str(c), "equivalent_to", "", ""]
                            )
                except:
                    pass

            # Handle the case where the is_a relation points to an "And" object:
            # Add to the and_nodes and relations.
            elif isinstance(c, owlready2.class_construct.And):
                and_counter += 1
                and_nodes[f"AND_{and_counter}"] = [f"AND_{and_counter}", "", "and"]
                if str(cls) in foodon_nodes:
                    fa_relations.append(
                        [str(cls), f"AND_{and_counter}", "is_a", "", ""]
                    )

                # For each of the subclasses in the "And" class, add a BLANK_x node,
                # and record the relation.
                # We also add the foodon object within the restriction to the list of foodon_nodes
                # and the "member_of" relations.
                for sub_c in c.Classes:
                    blank_counter += 1
                    blank_nodes[f"BLANK_{blank_counter}"] = [
                        f"BLANK_{blank_counter}",
                        "",
                        "blank",
                    ]
                    ba_relations.append(
                        [
                            f"BLANK_{blank_counter}",
                            f"AND_{and_counter}",
                            "member_of",
                            "",
                            "",
                        ]
                    )

                    # It looks like there can be more than one "layer" of "And" relations:
                    #
                    #                    foodon_node
                    #                       |
                    #                    AND_1
                    #                   /     \
                    #              AND_2     BLANK_1
                    #            /     \          |
                    #        BLANK_2   BLANK_3  foodon_node
                    #         |           |
                    #     foodon_node  foodon_node

                    # Ignoring this second (or deeper) layer for now.
                    if isinstance(sub_c, owlready2.class_construct.And):
                        continue

                    # Ignoring Or relations.
                    if isinstance(sub_c, owlready2.class_construct.Or):
                        continue

                    # Ignoring Not relations.
                    if isinstance(sub_c, owlready2.class_construct.Not):
                        continue

                    # Checking to make sure the restriction type is "SOME"
                    if sub_c.type == SOME:
                        restriction_type = "some"
                    elif sub_c.type == ONLY:
                        restriction_type = "only"

                    # Ignoring (seemingly) invalid restriction types.
                    elif isinstance(sub_c.type, list):
                        continue

                    if str(sub_c.value) in foodon_nodes:
                        bf_relations.append(
                            [
                                f"BLANK_{blank_counter}",
                                str(sub_c.value),
                                str(sub_c.property),
                                list(set(sub_c.property.label)),
                                restriction_type,
                            ]
                        )
                    if str(sub_c.property) in foodon_nodes:
                        node = [
                            str(sub_c.property),
                            list(set(sub_c.property.label)),
                            "FoodOn",
                        ]
                        foodon_nodes[str(sub_c.property)] = node

    with open(foodon_nodes_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(foodon_nodes_header)
        writer.writerows(foodon_nodes.values())

    with open(and_nodes_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(and_nodes_header)
        writer.writerows(and_nodes.values())

    with open(blank_nodes_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(blank_nodes_header)
        writer.writerows(blank_nodes.values())

    with open(ff_relations_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(ff_relations_header)
        writer.writerows(ff_relations)

    with open(fa_relations_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(fa_relations_header)
        writer.writerows(fa_relations)

    with open(ba_relations_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(ba_relations_header)
        writer.writerows(ba_relations)

    with open(bf_relations_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(bf_relations_header)
        writer.writerows(bf_relations)

In [10]:
create_nodes_relations(
    foodon,
    "foodon_nodes.csv",
    "and_nodes.csv",
    "blank_nodes.csv",
    "ff_relations.csv",
    "fa_relations.csv",
    "ba_relations.csv",
    "bf_relations.csv",
)

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_00002511

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_03315876



Use the following command to load the data into neo4j:

`../bin/neo4j-admin import --force --nodes=FoodOn=foodon_nodes.csv --nodes=And=and_nodes.csv --nodes=Blank=blank_nodes.csv --relationships=ff_relations.csv --relationships=fa_relations.csv --relationships=ba_relations.csv --relationships=bf_relations.csv`